# Monitor and Improve Existing Agent Performance

In this notebook, we focus on analyzing and improving the performance of existing agents. Agents may be performing at various levels—whether they are doing well, okay, or poorly—and their performance may change over time. The goal is to track their performance and provide targeted interventions to help them evolve into top-performing agents.

1. **Analyze Past and Current Performance Data**:  
   We begin by thoroughly analyzing the historical and current performance data available in the dataset. This will help us identify patterns, trends, and key performance metrics for each agent.

2. **Categorize Agents by Performance**:  
   Agents will be categorized based on their performance levels, such as High, Medium, and Low performers. This categorization allows us to tailor our approach to different agent groups and focus on the most impactful areas.

3. **Recommend Custom Interventions**:  
   For each performance group, we will recommend specific interventions that can help agents improve. These interventions will be customized based on the performance level and may include training programs, performance reviews, or other strategies.

4. **Track Progress Over Time**:  
   It is crucial to monitor the progress of agents over time to determine whether the interventions are effective. We will track their performance at regular intervals and assess if improvements are being made.


# Import libraries and load data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

# Load the data
file_path='datasets/train_storming_round.csv'
df = pd.read_csv(file_path)
    

# Preprocess the data for analysis

In [2]:
# Convert date columns to datetime format
date_columns = ['agent_join_month', 'first_policy_sold_month', 'year_month']
for col in date_columns:
    df[col] = pd.to_datetime(df[col])

# Calculate tenure (months since joining)
df['tenure_months'] = ((df['year_month'].dt.year - df['agent_join_month'].dt.year) * 12 + 
                        (df['year_month'].dt.month - df['agent_join_month'].dt.month))

# Calculate months to first sale
df['months_to_first_sale'] = ((df['first_policy_sold_month'].dt.year - df['agent_join_month'].dt.year) * 12 + 
                                (df['first_policy_sold_month'].dt.month - df['agent_join_month'].dt.month))

# Replace negative values with 0 (for cases where first_policy_sold_month is before agent_join_month)
df['months_to_first_sale'] = df['months_to_first_sale'].clip(lower=0)

# Calculate conversion rates
df['proposal_to_quotation_rate'] = df['unique_quotations'] / df['unique_proposal'].replace(0, 1)
df['quotation_to_policy_rate'] = df['new_policy_count'] / df['unique_quotations'].replace(0, 1)
df['overall_conversion_rate'] = df['new_policy_count'] / df['unique_proposal'].replace(0, 1)

# Calculate average policy value
df['avg_policy_value'] = df['ANBP_value'] / df['new_policy_count'].replace(0, 1)

# Calculate profit per policy
df['profit_per_policy'] = df['net_income'] / df['new_policy_count'].replace(0, 1)

# Calculate activity metrics
df['activity_rate_7days'] = df['unique_proposals_last_7_days'] / 7
df['activity_rate_15days'] = df['unique_proposals_last_15_days'] / 15
df['activity_rate_21days'] = df['unique_proposals_last_21_days'] / 21

# Calculate cash payment percentage
df['cash_payment_percentage'] = df['number_of_cash_payment_policies'] / df['number_of_policy_holders'].replace(0, 1) * 100

# Create performance features

In [3]:
performance_features = [
    'new_policy_count',              # Sales volume
    'ANBP_value',                    # Total sales value
    'net_income',                    # Profitability
    'avg_policy_value',              # Quality of sales
    'profit_per_policy',             # Efficiency
    'overall_conversion_rate',       # Sales efficiency
    'proposal_to_quotation_rate',    # Stage 1 conversion
    'quotation_to_policy_rate',      # Stage 2 conversion
    'unique_proposals_last_21_days', # Recent activity
    'activity_rate_21days',          # Consistency in prospecting
    'unique_customers'               # Customer reach
]

# Select only the relevant features for performance evaluation
performance_df = df[performance_features].copy()

# Handle missing values
performance_df.fillna(0, inplace=True)

# Classify agents using K-means clustering

In [4]:
# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(performance_df)

# Apply PCA for visualization (optional but helpful)
pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_features)
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

# Apply K-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(scaled_features)

# Analyze cluster centers to determine performance levels
cluster_centers = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_), 
                                columns=performance_features)

# Assign performance level to each cluster
# High performance: Higher policy count, ANBP, net income, etc.
# Calculate a composite score for each cluster
composite_scores = (
    cluster_centers['new_policy_count'] * 0.2 +
    cluster_centers['ANBP_value'] * 0.2 +
    cluster_centers['net_income'] * 0.2 +
    cluster_centers['overall_conversion_rate'] * 0.2 +
    cluster_centers['unique_proposals_last_21_days'] * 0.1 +
    cluster_centers['unique_customers'] * 0.1
)

# Rank clusters based on composite score
cluster_rankings = composite_scores.rank(ascending=False)
performance_mapping = {}

for cluster in range(3):
    if cluster_rankings[cluster] == 1:
        performance_mapping[cluster] = 'High'
    elif cluster_rankings[cluster] == 2:
        performance_mapping[cluster] = 'Medium'
    else:
        performance_mapping[cluster] = 'Low'

df['cluster'] = clusters
df['performance_level'] = df['cluster'].map(performance_mapping)


# Generate personalized intervention strategies 

In [5]:
# Create a new column for recommendations
df['intervention_strategy'] = ''

# Low performers
low_mask = df['performance_level'] == 'Low'

# Low conversion rates
low_conv_mask = (df['overall_conversion_rate'] < df['overall_conversion_rate'].quantile(0.25))
df.loc[low_mask & low_conv_mask, 'intervention_strategy'] += "1. Sales Training: Focus on improving conversion techniques and objection handling.\n"

# Low activity
low_activity_mask = (df['activity_rate_21days'] < df['activity_rate_21days'].quantile(0.25))
df.loc[low_mask & low_activity_mask, 'intervention_strategy'] += "2. Activity Management: Set daily prospecting targets and provide closer supervision.\n"

# New agents
new_agent_mask = (df['tenure_months'] <= 6)
df.loc[low_mask & new_agent_mask, 'intervention_strategy'] += "3. Mentorship Program: Pair with experienced agent for shadowing and guidance.\n"

# General recommendations for all low performers
df.loc[low_mask, 'intervention_strategy'] += "4. Weekly Performance Review: Schedule weekly one-on-one sessions to review metrics and provide feedback.\n"
df.loc[low_mask, 'intervention_strategy'] += "5. Product Knowledge: Complete refresher course on core products.\n"

# Medium performers
medium_mask = df['performance_level'] == 'Medium'

# Low high-value policies
low_value_mask = (df['avg_policy_value'] < df['avg_policy_value'].quantile(0.5))
df.loc[medium_mask & low_value_mask, 'intervention_strategy'] += "1. Upselling Training: Focus on identifying opportunities for premium products.\n"

# Inconsistent activity
inconsistent_mask = (df['unique_proposals_last_21_days'] - df['unique_proposals_last_7_days'] * 3).abs() > 5
df.loc[medium_mask & inconsistent_mask, 'intervention_strategy'] += "2. Consistency Program: Implement daily activity tracking and regular scheduling.\n"

# Limited customer base
limited_customers_mask = (df['unique_customers'] < df['unique_customers'].quantile(0.5))
df.loc[medium_mask & limited_customers_mask, 'intervention_strategy'] += "3. Networking Strategy: Provide resources for expanding customer base and referrals.\n"

# General recommendations for all medium performers
df.loc[medium_mask, 'intervention_strategy'] += "4. Specialized Product Training: Advanced training on high-margin products.\n"
df.loc[medium_mask, 'intervention_strategy'] += "5. Monthly Group Coaching: Join peer group sessions to share best practices.\n"

# High performers
high_mask = df['performance_level'] == 'High'

# Focus on retention
df.loc[high_mask, 'intervention_strategy'] += "1. Client Retention Program: Implement a structured follow-up system for existing clients.\n"

# Leadership development
df.loc[high_mask, 'intervention_strategy'] += "2. Leadership Development: Prepare for team leadership and mentoring roles.\n"

# Advanced sales techniques
df.loc[high_mask, 'intervention_strategy'] += "3. Advanced Sales Techniques: Training on complex products and high-net-worth client acquisition.\n"

# Recognition and motivation
df.loc[high_mask, 'intervention_strategy'] += "4. Recognition Program: Highlight achievements in company communications and events.\n"

# Career development
df.loc[high_mask, 'intervention_strategy'] += "5. Career Path Planning: Set long-term goals and development plan for advancement.\n"

# Create a progress tracker

In [6]:
# Key metrics to track
tracking_metrics = [
    'new_policy_count',
    'ANBP_value',
    'net_income',
    'overall_conversion_rate',
    'unique_proposals_last_21_days',
    'unique_customers'
]

# Create a baseline for current performance
progress_tracker = df.groupby(['agent_code', 'performance_level']).agg({
    'new_policy_count': 'mean',
    'ANBP_value': 'mean',
    'net_income': 'mean',
    'overall_conversion_rate': 'mean',
    'unique_proposals_last_21_days': 'mean',
    'unique_customers': 'mean'
}).reset_index()

# Rename columns to indicate they're baseline values
for metric in tracking_metrics:
    progress_tracker = progress_tracker.rename(columns={metric: f'{metric}_baseline'})

# Set improvement targets based on performance level
# Low performers: aim for 30% improvement
# Medium performers: aim for 20% improvement
# High performers: aim for 10% improvement
for metric in tracking_metrics:
    progress_tracker[f'{metric}_target'] = progress_tracker.apply(
        lambda x: x[f'{metric}_baseline'] * (1.3 if x['performance_level'] == 'Low' 
                                            else 1.2 if x['performance_level'] == 'Medium'
                                            else 1.1),
        axis=1
    )

# Add tracking timeframe (3-month intervention period)
today = datetime.now()
progress_tracker['intervention_start_date'] = today.strftime('%Y-%m-%d')
progress_tracker['first_checkpoint'] = (today + timedelta(days=30)).strftime('%Y-%m-%d')
progress_tracker['second_checkpoint'] = (today + timedelta(days=60)).strftime('%Y-%m-%d')
progress_tracker['final_evaluation'] = (today + timedelta(days=90)).strftime('%Y-%m-%d')

# Add columns for checkpoint data (to be filled later)
for checkpoint in ['first_checkpoint', 'second_checkpoint', 'final_evaluation']:
    for metric in tracking_metrics:
        progress_tracker[f'{metric}_{checkpoint}'] = np.nan
        
# Add column for improvement status (to be updated at each checkpoint)
progress_tracker['improvement_status'] = 'Pending'


# Create visualizations 

In [7]:

# Distribution of performance levels
perf_dist = df['performance_level'].value_counts()

# Average metrics by performance level
avg_metrics = df.groupby('performance_level').agg({
    'new_policy_count': 'mean',
    'ANBP_value': 'mean',
    'net_income': 'mean',
    'overall_conversion_rate': 'mean',
    'unique_customers': 'mean'
}).reset_index()

# Create plots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Performance Level Distribution', 'Average Policy Count by Performance Level',
                    'Average ANBP Value by Performance Level', 'Average Conversion Rate by Performance Level'),
    specs=[[{'type': 'domain'}, {'type': 'bar'}],
            [{'type': 'bar'}, {'type': 'bar'}]]
)

# Performance distribution pie chart
fig.add_trace(
    go.Pie(labels=perf_dist.index, values=perf_dist.values, name='Performance Distribution',
           marker=dict(colors=['red', 'gold', 'green'])),
    row=1, col=1
)

# Average policy count bar chart
fig.add_trace(
    go.Bar(x=avg_metrics['performance_level'], y=avg_metrics['new_policy_count'], name='Avg Policy Count',
            marker_color=['red', 'gold', 'green'], showlegend=False),
    row=1, col=2
)

# Average ANBP bar chart
fig.add_trace(
    go.Bar(x=avg_metrics['performance_level'], y=avg_metrics['ANBP_value'], name='Avg ANBP Value',
            marker_color=['red', 'gold', 'green'], showlegend=False),
    row=2, col=1
)

# Average conversion rate bar chart
fig.add_trace(
    go.Bar(x=avg_metrics['performance_level'], y=avg_metrics['overall_conversion_rate'], name='Avg Conversion Rate',
            marker_color=['red', 'gold', 'green'], showlegend=False),
    row=2, col=2
)

fig.update_layout(height=800, width=1000, title_text="Agent Performance Dashboard")

# Display results

In [8]:
# Display results
performance_summary = df.groupby('performance_level').agg({
    'agent_code': 'count',
    'new_policy_count': 'mean',
    'ANBP_value': 'mean',
    'net_income': 'mean',
    'overall_conversion_rate': 'mean'
}).rename(columns={'agent_code': 'count'})

print("Agent Performance Classification Summary:")
print(performance_summary)

# Save results to files
df.to_csv('agent_performance_classified.csv', index=False)
progress_tracker.to_csv('agent_progress_tracker.csv', index=False)

# Example of displaying strategies for a few agents from each category
print("\nExample Intervention Strategies:")
for level in ['Low', 'Medium', 'High']:
    example_agents = df[df['performance_level'] == level].head(1)
    for _, agent in example_agents.iterrows():
        print(f"\nAgent Code: {agent['agent_code']} (Performance Level: {level})")
        print(agent['intervention_strategy'])


Agent Performance Classification Summary:
                   count  new_policy_count    ANBP_value     net_income  \
performance_level                                                         
High                6994         27.700029  1.553551e+06  314265.227910   
Low                 1528          0.000000  0.000000e+00  224314.337042   
Medium              6786         17.176393  7.118090e+05  140013.897878   

                   overall_conversion_rate  
performance_level                           
High                              1.240545  
Low                               0.000000  
Medium                            1.464065  

Example Intervention Strategies:

Agent Code: 455ca878 (Performance Level: Low)
1. Sales Training: Focus on improving conversion techniques and objection handling.
2. Activity Management: Set daily prospecting targets and provide closer supervision.
4. Weekly Performance Review: Schedule weekly one-on-one sessions to review metrics and provide feedback.
